# HPO for Customer Churn Prediction with XGBoost
_**본 노트북은 1번 노트북에서 작업한 고객이탈예측에 하이퍼파라미터 최적화 작업을 수행합니다.**_


---
#### 주의 : 1번 노트북 작업을 완료후 실행해야 합니다.

---

## Contents

1. [Setup](#Setup)
1. [Train with HPO](#Train-with-HPO)
1. [Analyze result of HPO job](#Analyze-result-of-HPO-job)
1. [Analyze result of HPO parameters](#Analyze-result-of-HPO-parameters)
1. [Host](#Host)
  1. [Evaluate](#Evaluate)
  1. [Compare results](#Compare-results)

---

## Setup

SageMaker SDK를 로드하고 초기변수를 설정합니다. (1번 노트북의 첫단계와 동일합니다.)


In [1]:
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = "sagemaker/DEMO-xgboost-churn"

# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

필요한 파이썬 라이브러리를 import 합니다. 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
from sagemaker.inputs import TrainingInput
from sagemaker.serializers import CSVSerializer

1번 노트북에서 사용한 변수를 리스토어합니다.

In [3]:
%store -r s3_input_train s3_input_validation test_data predictions

In [4]:
print(s3_input_train.config.values())

dict_values([{'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-west-2-322537213286/sagemaker/DEMO-xgboost-churn/train', 'S3DataDistributionType': 'FullyReplicated'}}, 'csv'])


---
## Train with HPO

HPO 작업을 위해 XGBoost Estimator를 선언합니다. 먼저 XGBoost 컨테이너의 위치를 가져옵니다.

In [5]:
container = sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name, "latest")
display(container)

'433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest'

Estimator의 설정은 1번 단계와 거의 동일하나 몇가지가 달라졌습니다.
- hyperparameter를 세팅할 때 max_depth, etc, min_child_weight 값을 설정하지 않았습니다.
- xgb.fit()을 바로 실행하지 않았습니다. (대신 바로 다음 스텝에서 Hyperparameter tuning 작업의 매개변수로 전달됩니다.)

In [6]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sess,
)
xgb.set_hyperparameters(
    gamma=4,
    subsample=0.8,
    silent=0,
    objective="binary:logistic",
    num_round=100,
)


아래는 1번 노트북에서 사용한 코드입니다. 어떤 부분이 달려졌는지 비교해 보시기 바랍니다.

```
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sess,
)
xgb.set_hyperparameters(
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    silent=0,
    objective="binary:logistic",
    num_round=100,
)

xgb.fit({"train": s3_input_train, "validation": s3_input_validation})
```



### SageMaker tunner 선언

SageMaker Hypter parameter tunner를 선언합니다.
- objective_metric_name : HPO작업에서 최적화할 목표 매트릭을 설정
- hyperparameter_ranges : 탐색하고자하는 하이퍼파라미터와 해당 값의 범위를 지정
- max_jobs : 총 실행할 작업의 회수
- max_parallel_jobs : 동시에 실행할 작업의 회수

In [7]:
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

objective_metric_name = "validation:auc"

hyperparameter_ranges = {
    "eta": ContinuousParameter(0, 1),
    "min_child_weight": ContinuousParameter(1, 10),
    "max_depth": IntegerParameter(1, 10),
}

tuner = HyperparameterTuner(
    xgb, objective_metric_name, hyperparameter_ranges, max_jobs=20, max_parallel_jobs=3
)


### HPO 작업 실행
다음 셀을 실행하고 SageMaker 콘솔로 이동하여 Hyperparameter tuning jobs 메뉴를 클릭합니다. 작업이 제출되고 실행되는 것을 모니터링할 수 있습니다. (max_jobs와 max_parallel_jobs 설정이 HPO 작업실행에 어떤 영향을 미치는지 확인해 보세요.)

In [8]:
%%time
tuner.fit({"train": s3_input_train, "validation": s3_input_validation})

............................................................................................................................................................................................................................................................................................................................................................!
CPU times: user 1.16 s, sys: 122 ms, total: 1.28 s
Wall time: 29min 20s


## Analyze result of HPO job

HPO 작업이 완료되면 다음 코드를 이용하여 결과를 탐색해 봅니다.

분석 코드는 아래 SageMaker 공식예제로부터 일부 변수값 등을 변형하였습니다.
- https://github.com/aws/amazon-sagemaker-examples/blob/master/hyperparameter_tuning/analyze_results/HPO_Analyze_TuningJob_Results.ipynb


In [9]:
tuning_job_name=tuner.latest_tuning_job.job_name

boto3.client("sagemaker").describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name
)["HyperParameterTuningJobStatus"]


'Completed'

boto3 SDK를 이용하여 작업결과를 가져오고 `tuning_job_result` 로 저장하였습니다. 해당 오브젝트를 살펴보면 실행한 작업의 수 등 HPO작업의 설정 사항과 상태 등을 확인할 수 있습니다.


In [10]:
sage_client = boto3.Session().client("sagemaker")
# run this cell to check current status of hyperparameter tuning job
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name
)

status = tuning_job_result["HyperParameterTuningJobStatus"]
if status != "Completed":
    print("Reminder: the tuning job has not been completed.")

job_count = tuning_job_result["TrainingJobStatusCounters"]["Completed"]
print("%d training jobs have completed" % job_count)

is_minimize = (
    tuning_job_result["HyperParameterTuningJobConfig"]["HyperParameterTuningJobObjective"]["Type"] != "Maximize"
)
objective_name = tuning_job_result["HyperParameterTuningJobConfig"]["HyperParameterTuningJobObjective"]["MetricName"]


20 training jobs have completed


20번의 탐색 중 가장 높은 성능을 보인 작업을 살펴봅니다. 작업실행 시간등의 로그와 함께 탐색대상으로 지정했던 `eta`, `max_depth`, `min_child_weight` 에 대한 최적값을 확인할 수 있습니다. (`TunedHyperParameters`항목을 확인합니다.)

In [11]:
from pprint import pprint

if tuning_job_result.get("BestTrainingJob", None):
    print("Best model found so far:")
    pprint(tuning_job_result["BestTrainingJob"])
else:
    print("No training jobs have reported results yet.")


Best model found so far:
{'CreationTime': datetime.datetime(2021, 12, 4, 8, 8, 52, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'validation:auc',
                                                 'Value': 0.9807050228118896},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2021, 12, 4, 8, 12, 37, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:us-west-2:322537213286:training-job/xgboost-211204-0804-004-08937735',
 'TrainingJobName': 'xgboost-211204-0804-004-08937735',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2021, 12, 4, 8, 11, 45, tzinfo=tzlocal()),
 'TunedHyperParameters': {'eta': '0.09163922653817737',
                          'max_depth': '5',
                          'min_child_weight': '1.276436568789253'}}


## Analyze result of HPO parameters

SageMaker의 HyperparameterTuningJobAnalytics를 이용하여 하이퍼파라미터 탐색작업에서 실행한 실험의 결과를 살펴봅니다.

In [12]:
tuning_result = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)
full_df = tuning_result.dataframe()

if len(full_df) > 0:
    df = full_df[full_df["FinalObjectiveValue"] > -float("inf")]
    if len(df) > 0:
        df = df.sort_values("FinalObjectiveValue", ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest": min(df["FinalObjectiveValue"]), "highest": max(df["FinalObjectiveValue"])})
        pd.set_option("display.max_colwidth", -1)  # Don't truncate TrainingJobName
    else:
        print("No training jobs have reported valid results yet.")

df

Number of training jobs with valid objective: 20
{'lowest': 0.9398990273475647, 'highest': 0.9807050228118896}


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


,eta,max_depth,min_child_weight,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
16,0.091639,5.0,1.276437,xgboost-211204-0804-004-08937735,Completed,0.980705,2021-12-04 08:11:45+00:00,2021-12-04 08:12:37+00:00,52.0
7,0.200770,4.0,5.971817,xgboost-211204-0804-013-ab276950,Completed,0.980581,2021-12-04 08:23:41+00:00,2021-12-04 08:24:33+00:00,52.0
6,0.246669,10.0,4.130685,xgboost-211204-0804-014-15f97d30,Completed,0.980241,2021-12-04 08:24:22+00:00,2021-12-04 08:25:14+00:00,52.0
15,0.091639,5.0,1.456437,xgboost-211204-0804-005-110a3493,Completed,0.979896,2021-12-04 08:11:49+00:00,2021-12-04 08:12:41+00:00,52.0
9,0.096949,5.0,7.492875,xgboost-211204-0804-011-f6fcb6ee,Completed,0.979760,2021-12-04 08:20:11+00:00,2021-12-04 08:21:13+00:00,62.0
8,0.215022,4.0,9.999680,xgboost-211204-0804-012-0994e863,Completed,0.979540,2021-12-04 08:20:31+00:00,2021-12-04 08:21:24+00:00,53.0
3,0.044612,9.0,1.579934,xgboost-211204-0804-017-9dfb91b9,Completed,0.979048,2021-12-04 08:28:13+00:00,2021-12-04 08:30:10+00:00,117.0
5,0.337225,4.0,1.619210,xgboost-211204-0804-015-13d55ff6,Completed,0.978912,2021-12-04 08:24:29+00:00,2021-12-04 08:25:35+00:00,66.0
18,0.173270,7.0,2.663128,xgboost-211204-0804-002-7b738a8d,Completed,0.978712,2021-12-04 08:07:49+00:00,2021-12-04 08:08:53+00:00,64.0
2,0.054612,9.0,1.669934,xgboost-211204-0804-018-dc9ed238,Completed,0.978576,2021-12-04 08:28:28+00:00,2021-12-04 08:29:26+00:00,58.0


그래프를 통해 작업별 실행한 결과를 확인해 보겠습니다. 그래프상의 관측값에 마우스를 위치시키면 해당 실험이 하이퍼파라미터 범위 중 어떤 값을 사용하였고 그 때의 매트릭이 어떻게 관측되었는지 확인할 수 있습니다.

In [13]:
import bokeh
import bokeh.io

bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool


class HoverHelper:
    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append((k, "@{%s}" % k))

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools="pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset"):
        return [self.hovertool(), standard_tools]


hover = HoverHelper(tuning_result)

p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type="datetime")
p.circle(source=df, x="TrainingStartTime", y="FinalObjectiveValue")
show(p)

Loading BokehJS ...

유사한 방법으로 탐색대상으로 설정한 하이퍼파라미터별로 매트릭의 결과에 어떤 영향을 미쳤는지 확인해 봅니다.

In [14]:
ranges = tuning_result.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get("Values"):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0

        vals = hp_range["Values"]
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args["x_range"] = vals

    # Now plot it
    p = figure(
        plot_width=500,
        plot_height=500,
        title="Objective vs %s" % hp_name,
        tools=hover.tools(),
        x_axis_label=hp_name,
        y_axis_label=objective_name,
        **categorical_args,
    )
    p.circle(source=df, x=hp_name, y="FinalObjectiveValue")
    figures.append(p)
show(bokeh.layouts.Column(*figures))

---
## Host

HPO작업을 통해 찾아낸 최적의 하이퍼파라미터를 이용하여 모델을 생성하고 호스팅 엔드포인트로 배포합니다. 


In [15]:
hpo_predictor = tuner.deploy(
    initial_instance_count=1, instance_type="ml.m4.xlarge", serializer=CSVSerializer()
)


2021-12-04 08:12:37 Starting - Preparing the instances for training
2021-12-04 08:12:37 Downloading - Downloading input data
2021-12-04 08:12:37 Training - Training image download completed. Training in progress.
2021-12-04 08:12:37 Uploading - Uploading generated training model
2021-12-04 08:12:37 Completed - Training job completed
------!

### Evaluate

1번 노트북에서 실행했던 것과 유사한 방법으로 테스트데이터에 대한 예측을 실행하과 1번 작업의 결과와 비교해 보겠습니다.


In [16]:
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ""
    for array in split_array:
        predictions = ",".join([predictions, hpo_predictor.predict(array).decode("utf-8")])

    return np.fromstring(predictions[1:], sep=",")


predictions_hpo = predict(test_data.to_numpy()[:, 1:])

### Compare results

이번에는 sklearn에서 제공하는 classification_report 기능을 이용하여 결과를 확인하겠습니다. 편의상 0.5를 컷오프로 지정하였습니다. 


In [17]:
from sklearn.metrics import classification_report
print(classification_report(test_data.iloc[:, 0], np.round(predictions)))

              precision    recall  f1-score   support

           0       0.96      0.92      0.94       253
           1       0.92      0.96      0.94       247

    accuracy                           0.94       500
   macro avg       0.94      0.94      0.94       500
weighted avg       0.94      0.94      0.94       500



다음은 HPO로 찾아낸 하이퍼파라미터로 작업한 결과입니다. 전체적으로 이전 모델의 성능과 유사하나 0번 분류에 대한 precision과 recall 값이 조금 상승하였습니다.  

In [18]:
from sklearn.metrics import classification_report
print(classification_report(test_data.iloc[:, 0], np.round(predictions_hpo)))

              precision    recall  f1-score   support

           0       0.96      0.93      0.95       253
           1       0.93      0.96      0.95       247

    accuracy                           0.95       500
   macro avg       0.95      0.95      0.95       500
weighted avg       0.95      0.95      0.95       500



### (Optional) Clean-up

모든 작업을 완료하였다면 추가 요금발생을 막기 위해 아래 셀을 실행하십시오. 아래 코드는 생성한 호스팅 엔드포인트를 제거합니다. 

In [19]:
hpo_predictor.delete_endpoint()